# Yolo object detection

In [2]:
# Imports

import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, _verbose=False)

Using cache found in C:\Users\matth/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-11-21 Python-3.8.0 torch-1.11.0+cpu CPU


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
# Plot bounding boxes
def plot_boxes(results, frame, model):

	# Get labels and coördinates
	labels, cord = results

	# Get image shape
	x_shape, y_shape = frame.shape[1], frame.shape[0]

	# Loop
	count = 0
	for i in range(len(labels)):

		# Get coordinates of bounding box
		row = cord[i]

		# Supress
		if row[4] < 0.5: continue

		# Get pixel coordinates
		x1 = int(row[0]*x_shape)
		y1 = int(row[1]*y_shape)
		x2 = int(row[2]*x_shape)
		y2 = int(row[3]*y_shape)

		# Color of bounding box
		bgr = (0, 255, 0) 

		# Get the name of label index
		classes = model.names 

		# If a person
		if int(labels[i]) == 0: count += 1

		# Get font for the label
		label_font = cv2.FONT_HERSHEY_SIMPLEX 

		# Plot the boxes
		cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2) 

		# Put a label over box.
		cv2.putText(frame, classes[int(labels[i])], (x1, y1), label_font, 0.9, bgr, 2) 

	# Count people
	cv2.putText(frame, "Number of visible visitors: " + str(count), (int(np.shape(frame)[1]/4), 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2, cv2.LINE_AA)

	return frame

# Score frame
def score_frame(frame, model):

	# Check device
	device = 'cuda' if torch.cuda.is_available() else 'cpu'
	print(device)
	model.to(device)

	# Convert frame to tensor
	frame = [torch.tensor(frame)]

	# Run through network
	results = model(frame)

	# Retrieve labels en coordinates
	labels = results.xyxyn[0][:, -1].numpy()
	cord = results.xyxyn[0][:, :-1].numpy()

	return labels, 

In [ ]:
# Load image
img = cv2.imread('data/smiling_person.jpg')

# Reshape image
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# Score the frame
results = score_frame(gray, model) 

# Plot the boxes
image = plot_boxes(results, img, model) 

# Show result
plt.imshow(img)
plt.show()